Copyright 2021-2023 @ Shenzhen Bay Laboratory & Peking University & Huawei Technologies Co., Ltd

This code is a part of MindSPONGE:
MindSpore Simulation Package tOwards Next Generation molecular modelling.

MindSPONGE is open-source software based on the AI-framework:
MindSpore (https://www.mindspore.cn/)

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.

You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.

See the License for the specific language governing permissions and
limitations under the License.

MindSPONGE advanced tutorial 01: Collective variables (CVs), metrics and analyse.

In [1]:
from mindspore import context
from mindsponge import Sponge
from mindsponge import ForceField
from mindsponge import set_global_units
from mindsponge import Protein
from mindsponge import UpdaterMD
from mindsponge.optimizer import SteepestDescent
from mindsponge.control import VelocityVerlet
from mindsponge.callback import WriteH5MD, RunInfo
from mindsponge.control import Langevin
from mindsponge.function import VelocityGenerator
from mindsponge.colvar import Torsion

[WARNING] ME(280321:140287255590144,MainProcess):2023-03-25-14:24:54.815.123 [mindspore/run_check/_check_version.py:75] Can not found cuda libs, please confirm that the correct cuda version has been installed, you can refer to the installation guidelines: https://www.mindspore.cn/install


In [2]:
context.set_context(mode=context.PYNATIVE_MODE, device_target="GPU")
set_global_units('nm', 'kj/mol')

In [3]:
system = Protein(pdb='alad.pdb')
potential = ForceField(system, 'AMBER.FF14SB')

In [4]:
phi = Torsion([4, 6, 8, 14])
psi = Torsion([6, 8, 14, 16])

In [5]:
min_opt = SteepestDescent(system.trainable_params(), 1e-7)
mini = Sponge(system, potential, min_opt, metrics={'phi': phi, 'psi': psi})

[MindSPONGE] Using fully connected neighbour list (not updated).


In [6]:
cv = mini.analyse()
for k, v in cv.items():
    print(k, v)


phi [[3.1415927]]
psi [[3.1415927]]


In [7]:
run_info = RunInfo(10)
mini.run(100, callbacks=[run_info])

[MindSPONGE] Started simulation at 2023-03-25 14:25:06
[MindSPONGE] Step: 0, E_pot: -55.75643, phi: -3.140886, psi: -3.1410766
[MindSPONGE] Step: 10, E_pot: -58.41481, phi: -3.1339903, psi: -3.1368215
[MindSPONGE] Step: 20, E_pot: -60.197777, phi: -3.1276336, psi: -3.1337032
[MindSPONGE] Step: 30, E_pot: -61.60443, phi: -3.1219037, psi: -3.1313035
[MindSPONGE] Step: 40, E_pot: -62.77914, phi: -3.1167603, psi: -3.1294105
[MindSPONGE] Step: 50, E_pot: -63.786407, phi: -3.1121268, psi: -3.1279025
[MindSPONGE] Step: 60, E_pot: -64.6644, phi: -3.107924, psi: -3.126701
[MindSPONGE] Step: 70, E_pot: -65.43922, phi: -3.1040812, psi: -3.125749
[MindSPONGE] Step: 80, E_pot: -66.13011, phi: -3.100538, psi: -3.125002
[MindSPONGE] Step: 90, E_pot: -66.7513, phi: -3.0972445, psi: -3.1244242
[MindSPONGE] Finished simulation at 2023-03-25 14:25:10
[MindSPONGE] Simulation time: 3.51 seconds.
--------------------------------------------------------------------------------


In [8]:
cv = mini.analyse()
for k, v in cv.items():
    print(k, v)

phi [[-3.0944598]]
psi [[-3.124025]]


In [9]:
vgen = VelocityGenerator(300)
velocity = vgen(system.coordinate.shape, system.atom_mass)
opt = UpdaterMD(
    system,
    integrator=VelocityVerlet(system),
    thermostat=Langevin(system, 300),
    time_step=1e-3,
    velocity=velocity
)

In [10]:
md = Sponge(system, potential, opt, metrics={'phi': phi, 'psi': psi})

[MindSPONGE] Using fully connected neighbour list (not updated).


In [11]:
cb_h5md = WriteH5MD(system, 'tutorial_a01.h5md', save_freq=10)
md.run(1000, callbacks=[run_info, cb_h5md])

[MindSPONGE] Started simulation at 2023-03-25 14:25:15
[MindSPONGE] Step: 0, E_pot: -67.313965, E_kin: 75.75505, E_tot: 8.441086, Temperature: 289.24567, phi: -3.0934665, psi: -3.1182637
[MindSPONGE] Step: 10, E_pot: -39.731663, E_kin: 52.939987, E_tot: 13.208324, Temperature: 202.13388, phi: -3.0182748, psi: -3.0506635
[MindSPONGE] Step: 20, E_pot: -34.170845, E_kin: 47.605576, E_tot: 13.434731, Temperature: 181.76619, phi: -2.951082, psi: -3.0755272
[MindSPONGE] Step: 30, E_pot: -38.478653, E_kin: 56.822617, E_tot: 18.343964, Temperature: 216.95842, phi: -2.9615834, psi: -3.1192112
[MindSPONGE] Step: 40, E_pot: -23.734085, E_kin: 51.823418, E_tot: 28.089333, Temperature: 197.87062, phi: -2.9811082, psi: 3.0580652
[MindSPONGE] Step: 50, E_pot: -35.640747, E_kin: 58.800564, E_tot: 23.159817, Temperature: 224.51056, phi: -2.897594, psi: 2.9556272
[MindSPONGE] Step: 60, E_pot: -35.642273, E_kin: 61.218464, E_tot: 25.57619, Temperature: 233.74251, phi: -2.759038, psi: 2.9617794
[MindSPONG

In [12]:
cv = md.analyse()
for k, v in cv.items():
    print(k, v)

phi [[-2.2645469]]
psi [[2.712899]]
